# SemanticModels.jl: Not Just Another Modeling Framework
*James Fairbanks*

Galois Inc, Aug 24, 2019

<center>
    <img src="src/img/semanticmodels_jl.dot.svg"></img>
</center>

- Teaching computers to do science
- Project Repo [github.com/jpfairbanks/SemanticModels.jl](github.com/jpfairbanks/SemanticModels.jl)

## Scientific Modeling

- Scientists make models out of math
- Models need to be implemented as code
- When you change the math, you need to change the code!


## Science as nested optimization

1. Observe Data $x, y$
2. Hypothesize governing laws $H$
3. Pick best paramters $h^* \in H$ with regression:
$$\underset{h\in {H}}{\operatorname{argmin}}\ell(h(x), y)$$ 

Institutional process of discovery is

$$\underset{{H}\in \mathcal{M}}{\operatorname{argmax}} expl(h^*)$$ where *expl* is the explanatory power of a class of models $H$. 

- The explanatory power is some combination of 
    - generalization,
    - parsimony,
    - and consistency with the fundamental principles of the field.

## Modeling Frameworks

Most frameworks are designed before the models are written

| Framework | Math | Input Specification  | Description |
|-----------|------|----------|-------------|
| <img width="50px" src="https://www.mathworks.com/content/dam/mathworks/mathworks-dot-com/images/ico_membrane_128.png"> Matlab/Scipy</img> | x = A\b | BLAS + scripting | Sci/Eng math is all BLAS| 
| <img width="50px" src="https://upload.wikimedia.org/wikipedia/commons/thumb/2/20/Mathematica_Logo.svg/1920px-Mathematica_Logo.svg.png"> Mathematica</img> |$p(x)=0$|Symbolic Math Expressions| Computer Algebra Systems|
| <img width="50px" src="https://mc-stan.org/docs/2_18/stan-users-guide/img/logo-tm.png" alt="Stan Logo">Stan</img> | $ y \sim \mathcal{N}(x \beta + \alpha, \sigma)$ | StanML | Bayesian Inference|
| <img src="https://camo.githubusercontent.com/31d60f762b44d0c3ea47cc16b785e042104a6e03/68747470733a2f2f7777772e6a756c69616f70742e6f72672f696d616765732f6a756d702d6c6f676f2d776974682d746578742e737667" alt="jump"></img> | $\min_{x\perp C(x)} f(x)$ | AMPL based DSL| Optimization Problems |
| ![TensorFlow](https://www.gstatic.com/devrel-devsite/va3a0eb1ff00a004a87e2f93101f27917d794beecfd23556fc6d8627bba2ff3cf/tensorflow/images/lockup.svg) | $y\approx f(x)$ | TF.Graph| Deep Learning|
| <img width="75px" src="http://aske.gtri.gatech.edu/docs/latest/img/semanticmodels_jl.dot.svg">SemanticModels.jl</img> | All Computable Domains| Julia Programs | $Models \subset Code$|

SemanticModels is a post hoc modeling framework

# What is a Modeling Framework?

1. Objects: $X,Y,Z$
2. Models: $f:X\rightarrow Y$, $g: Y\rightarrow Z$
3. Composition: $g\circ f: X\rightarrow Z$ 
4. Combination: $f\otimes g: X\otimes Y \rightarrow Y\otimes Z$
5. Executability: $\texttt{foo} = eval(f) \implies \texttt{foo(x::X)::Y}$ runs the model

## Statistical / ML models are accurate

Fitting curves to data is good, but doesn't explain the data.

## Scientific Models are Mechanistic
Mechanistic models are more explainable than black box or statistical models. They posit driving forces and natural laws
that drive the evolution of systems over time.

We call these *simulations* when necessary to distinguish from *model*

- Benefits: more explainable, more generalizable
- Cons: lower Accuracy, less flexible

## Mathematical Diagrams are pervasive
<center><img src="talks/bayesian_models.png">Two perspectives on Bayesian Networks (Jacobs, Kissinger, and Zanasi, 2019)</img></center>

# Scientists love diagrams
<center>
<img src="src/img/mathbio/SIS_otto.png">The SIS Model (figure from A Biologist's guide to Mathematical Modeling in Ecology and Evolution By Sarah Otto and Troy Day 2007) 
</img>
</center>

## SIR model of disease
<center>
    <img src="src/img/cd/tikzit/sir_petri+ode.png" width="80%">The SIR model shown with equivalent Petri net and system of ODEs </img>
</center>

## SIR predicts real diseases

<center>
    <img src="src/img/srep42594-f1-cropped.png">Ebola Outbreak Data</img>
</center>

- (a) Cumulative number of infected individuals as a function of time (day) for the three countries Guinea, Liberia and Sierra Leone. 

- A Khalequea, and P Senb, "An empirical analysis of the Ebola outbreak in West Africa" 2017

## ODE Based Implementation

This is a "real world" implementation of SIR modeling in Julia taken from Epirecipes Cookbook (Simon Frost)

```julia
module SIRModel
using DifferentialEquations

function sir_ode(du, u, p, t)
    #Infected per-Capita Rate
    β = p[1]
    #Recover per-capita rate
    γ = p[2]
    #Susceptible Individuals
    S = u[1]
    #Infected Individuals
    I = u[2]
    du[1] = -β * S * I
    du[2] = β * S * I - γ * I
    du[3] = γ * I
end

#Param = (Infected Per Capita Rate, Recover Per Capita Rate)
param = [0.1,0.05]
#Initial Params = (Susceptible Individuals, Infected by Infected Individuals)
init = [0.99,0.01,0.0]
tspan = (0.0,200.0)

sir_prob = ODEProblem(sir_ode, init, tspan, param)

sir_sol = solve(sir_prob, saveat = 0.1);
```

### Agent based simulation


```julia
""" Agent Models is a hypothetical ABM framework"""
module AgentModels

abstract type AgentModel end
mutable struct StateModel{T} <: AgentModel
    states::Vector{Symbol}
    data::T
    events::Vector{Function}
    rates::Vector{Number}
end

solve(m::StateModel) = return thesolution(m)
end
```

```julia
using AgentModels #hypothetical ABM framework

function main(nsteps)
    function infection(s)
        if s.S > 0 && s.I > 0
            s.S -= 1
            s.I -= 1
            s.I += 2
        end
    end
    function recovery(s)
        if s.I > 0
            s.I -= 1
            s.R += 1
        end
    end
    states = [:S, :I, :R]
    a = zeros(Int, states)
    ρ = 0.5 + randn(Float64)/4 # chance of recovery
    μ = 0.5 # chance of immunity
    T = [infection, recovery]
    prob = StateModel(states, a, T, [ρ, μ])
    soln = solve!(sam, nsteps)
    return soln
end
```

## Julia Programs as a Category

1. $Ob(C) = Types$
2. $Hom_C(S,T) = S\rightarrow T$ functions

![Julia Types SIR](src/img/types_sir.dot.png)

## Model Augmentation as a Lens

We want scientists to program using lenses
<center>
    <img src="src/img/cd/model_lens.png">Module Augmentation as a Lens</img>
</center>

*What should the $M_1, M_2$ be*?

## A Story of an Infectious Disease
<center>
<img src="./src/img/mathbio/wiring/seir.svg"></img>
</center>

## A Story of an Infectious Disease

<center>
<img src="./src/img/mathbio/wiring/seird.svg"></img>
</center>

## A Story of an Infectious Disease

<center>
<img src="./src/img/mathbio/wiring/seirds.svg"></img>
</center>

# Wiring Diagrams are a universal syntax
<center>
<img src="./talks/seirds_wire_petri.png">Associate all wires with the same label to get a petri net</img>
</center>

## Petrinet SIR Model

```julia
using Petri

function main()
    @variables S, I, R
    N = +(S,I,R)

    Δ = [(S+I, 2I),
         (I, R)]

    m = Petri.Model(Δ)
    p = Petri.Problem(m, SIRState(100, 1, 0), 50)
    soln = Petri.solve(p)
    (p, soln)
end
p, soln = main()

```

## Statistical Models

Statistical Data Analysis / ML workflows form a modeling framework

```julia
using LsqFit
function f(x, β) 
    return β[3] .* x.^2 .+ β[2] .* x .+ β[1]
end

function main(X_train, y_train, X_test, y_test)
    a₀ = [1.0, 1.0]
    βhat = curve_fit(f, X_train, y_train, a₀)
    yhat = f(X_test, βhat)
    r = loss(yhat, y_test)
    return r
end

main()
```

## Statistical Models

Statistical Data Analysis / ML workflows form a modeling framework

<center><img src="src/img/workflow/img/validation.svg"></img></center>

## Category Theory

CT is the mathematics of structure preserving maps. Every field of math has a notion of *homomorphism* where two objects
in that category *have similar structure*

1. Sets, Groups, Fields, Rings
2. Graphs
3. Databases

CT is the study of structure in its most general form.

## Categories as Objects and Morphisms

1. Categories have Objects, $ Ob(C) $
2. Morphisms are relations between objects $Hom_C(x,y)$
3. There is always an identity morphism $id_x \in Hom_C(x,y)$
4. Morphisms Compose $f\in Hom_C(x,y), g\in Hom_C(y,z)$ then $g\circ f \in Hom_C(x,z)$ 


## Graphs as Categories

### Each graph is a category
- $ G = (V,E) $
- $Ob(G) = V$
- $Hom_G(v,u) = \{(v\leadsto u) \in E\}$

<!--\{e \mid e\in E, src(e)=v \land dst(e)=u\}$ -->

### The category of graphs

- Graph Homomorphism $f: G\to H$ st $(v\leadsto u) \in G \implies (f(v) \leadsto f(u)) \in H$
- $Ob(Graph)$ is the set of all graphs
- $Hom_{Graph}(G,H)$ is the set of all graph homomorphisms $G$ to $H$



## Semantic Models Applies Category Theory

A novel modeling environment that builds and manipulates models in this category theory approach.

1. We take general code as input
2. Highly general and extensible framework
3. Goal: Transformations are compositional

## Converting Models between Categories
Models can be represented in different categories, for example, SIR as an OLOG.
![An SIR model structure](src/img/olog_sir.dot.svg)

## Type Graphs
1. The TypeGraph of a Julia Program tells you a lot about it
2. Computers are good at type checking
3. Can we embed our modeling semantics into the type system?


![Julia Types SIR](src/img/types_sir.dot.png)


![Resolving Type Ambiguities](src/img/type_ambig_resolve.png)

# Structural Model Changes
Modifying models using a Grammar of rewrite rules.

Reasoning by analogy


<img src="src/img/cd/rewrite_loop.png" width="80%">Double Push Outs over structured cospans (figure from Cicala, 2019) 
</img>


## Petrinet Model

```julia
using Petri

function main()
    @variables S, I, R
    N = +(S,I,R)

    Δ = [(S+I, 2I),
        (I, R),]

    m = Petri.Model(Δ)
    p = Petri.Problem(m, SIRState(100, 1, 0), 50)
    soln = Petri.solve(p)
    (p, soln)
end
p, soln = main()
```


## SIR $\rightarrow$  SIRS
![SIR model](src/img/cd/tikzit/rule_sir_sirs.png)

## SIRS model as code

```julia
using Petri

function main()
    @variables S, I, R
    N = +(S,I,R)

    Δ = [(S+I, 2I),
        (I, R),
        (R, S)]

    m = Petri.Model(Δ)
    p = Petri.Problem(m, SIRState(100, 1, 0), 50)
    soln = Petri.solve(p)
    (p, soln)
end
p, soln = main()
```


![An more refined ABM](src/img/cd/tikzit/sir_dpo.png)

## Software Interface for Rewriting Models
```julia
states = [S, I, R]

sir = Petri.Model(states,[(I, R), (S+I, 2I)])
ir = Petri.Model(states, [(I, R)])
seir = Petri.Model(states, [(I, R), (S+I, I+E), (E, I)])
rule = Span(sir, ir, seir)

# the root of the bottom of DPO
irs = Petri.Model(states, [(I, R), (R, S)])

sirs, seirs = solve(DPOProblem(rule, irs))
```


## Rewriting Models is a modeling framework

<center>
    <img src="src/img/workflow/img/rewrite.svg">
    We can recursively define a modeling framework for modeling frameworks!
    </img>
</center>

## Rewriting Models is a modeling framework
You can build up complex rewriting operations from simple ones.
<center>
    <img src="src/img/workflow/img/rewrite_twice.svg">
    A diagram of applying two rewrite rules.
    </img>
</center>

## SEIRS Model as Declarative Code

```julia
using Petri

function SEIRSmain()
    @variables S, E, I, R
    N = +(S,E,I,R)

    Δ = [(S+I, I+E),
         (E, I),
         (I, R),
         (R, S) 
    ]

    m = Petri.Model(Δ)
    p = Petri.Problem(m, SEIRState(100, 0, 1, 0), 150)
    soln = Petri.solve(p)
    (p, soln)
end
p, soln = SEIRSmain()

```

## SEIRS Model as Imperative Code
```julia
 :(##δ#754(state) = begin
          begin
              begin
                  state.I > 0 || return nothing
                  state.I -= 1
              end
              state.R += 1
          end
      end)                                                                                                                                                        
 :(##δ#755(state) = begin
          begin
              begin
                  state.S > 0 || return nothing
                  state.I > 0 || return nothing
                  state.S -= 1
                  state.I -= 1
              end
              begin
                  state.I += 1
                  state.E += 1
              end
          end
      end)
 :(##δ#756(state) = begin
          begin
              begin
                  state.E > 0 || return nothing
                  state.E -= 1
              end
              state.I += 1
          end
      end)                                                                                                                                                        
 :(##δ#757(state) = begin
          begin
              begin
                  state.R > 0 || return nothing
                  state.R -= 1
              end
              state.S += 1
          end
end)                            
```

## Compiler / Disassembler
<center>
    <img src="src/img/cd/tikzit/model_lens_2.png">Module Augmentation as a Lens</img>
</center>


## SEIRS Model as x86 Assembly
A tiny portion of the model!

```
	.section	__TEXT,__text,regular,pure_instructions
; Function ##δ#788 {
; Location: rewrite.jl:127
; Function getproperty; {
; Location: rewrite.jl:148
	decl	%eax
	movl	8(%esi), %eax
;}
; Function >; {
; Location: operators.jl:286
; Function <; {
; Location: int.jl:49
	decl	%eax
	testl	%eax, %eax
;}}
	jle	L37
; Function -; {
; Location: int.jl:52
	decl	%eax
	addl	$-1, %eax
;}
; Function setproperty!; {
; Location: sysimg.jl:19
	decl	%eax
	movl	%eax, 8(%esi)
;}
; Function getproperty; {
; Location: sysimg.jl:18
	decl	%eax
	movl	16(%esi), %eax
;}
; Function +; {
; Location: int.jl:53
	decl	%eax
	addl	$1, %eax
;}
; Function setproperty!; {
; Location: sysimg.jl:19
	decl	%eax
	movl	%eax, 16(%esi)
;}
	decl	%eax
	movl	%eax, (%edi)
	movb	$2, %dl
	xorl	%eax, %eax
	retl
L37:
	movb	$1, %dl
	xorl	%eax, %eax
; Location: rewrite.jl:127
	retl
	nopw	(%eax,%eax)
;}
```

## Compiling Petri.Model to ODEProblem
```julia
f(du, state, p, t) = begin
    du.S = (-p[1] * (((state.β * state.S) * state.I) 
                      / ((state.S + state.I) + state.R))) 
            + p[3] * (state.μ * state.R)
    
    du.I = (p[1] * (((state.β * state.S) * state.I) 
                      / ((state.S + state.I) + state.R))) 
            + -p[2] * (state.γ * state.I)
    
    du.R = ((p[2] * (state.γ * state.I)) 
            + -p[3] * (state.μ * state.R))
end

function main()
  param = [0.1,0.05]
  init = [0.99,0.01,0.0]
  tspan = (0.0,200.0)

  prob = ODEProblem(f, init, tspan, param)

  sol = solve(prob);
end

```

## SIRD Model

```julia
using Petri

function SIRDmain()
    @variables S, I, R, D
    N = +(S,E,I,R, D)

    Δ = [(S+I, 2I),
         (I, R),
         (R, S),
         (I, D)
    ]

    m = Petri.Model(Δ)
    p = Petri.Problem(m, SIRDState(100, 1, 0, 0), 150)
    soln = Petri.solve(p)
    (p, soln)
end
p, soln = SIRDmain()
```

# Modeling Frameworks

![](src/img/mathbio/final_juliacon.png)

# Lenses + Rewriting

<center><img src="talks/lens+dpo.svg"></img></center>

# Conclusion

1. SemanticModels.jl github.com/jpfairbanks/SemanticModels.jl is a foundational technology for teaching machines to reason about scientific models

2. SemanticModels.jl combines DPO rewriting with Lenses for model augmentation **for science!**

3. $SemanticModels = Codification \circ Categorification \circ Science $

## Open Questions

1. Which scientific modeling frameworks can we represent? 
2. How can we compute rewriting for general frameworks?
3. What other modeling activities can we formalize?


# Acknowledgements
![Acknowledgements](src/img/Acknowledgements_ASKE.png)